![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np


from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

In [3]:
# Carga de datos de archivos .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [4]:
print(dataTraining.dtypes)

Price       int64
Year        int64
Mileage     int64
State      object
Make       object
Model      object
dtype: object


## Codificar variables categóricas
Relizar preprocesamiento para crear variables predictoras:

In [5]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
dataTraining['State'] = pd.factorize(dataTraining.State)[0]
dataTraining['Make'] = pd.factorize(dataTraining.Make)[0]
dataTraining['Model'] = pd.factorize(dataTraining.Model)[0]
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,0,0,0
1,37895,2015,20578,1,1,1
2,18430,2012,83716,2,2,2
3,24681,2014,28729,1,3,3
4,26998,2013,64032,3,0,0


In [9]:
#Particion de x-y datos training
XTotal = dataTraining.drop(["Price"], axis = 1)  
yTotal = dataTraining.filter(["Price"], axis = 1)

# particion de training para calculo de MSE
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.20, random_state=0)

## Entrenar y guardar el modelo

In [10]:
# Definición de modelo de clasificación Random Forest
grb = GradientBoostingRegressor()

In [11]:
# Entrenamiento del modelo de clasificación Random Forest
grb.fit(XTrain, yTrain)

GradientBoostingRegressor()

In [12]:
# Exportar modelo a archivo binario .pkl
joblib.dump(grb, 'model_deployment_proyecto/pricecar_grb.pkl', compress=3)

['model_deployment_proyecto1/pricecar_grb.pkl']

In [ ]:
# Importar modelo y predicción
from model_deployment_proyecto.m09_model_deployment_proyecto import predict_price

# Predicción de probabilidad de que un link sea phishing
#predict_price('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [ ]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [ ]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Price Car Prediction API',
    description='Price Car Prediction API')

ns = api.namespace('predict', 
     description='Price Car Regressor')
   
parser = api.parser()

parser.add_argument(
    'YEAR', 
    type=int, 
    required=True, 
    help='Year of the car', 
    location='args')

parser.add_argument(
    'STATE', 
    type=str, 
    required=True, 
    help='state where the car was sold', 
    location='args')

parser.add_argument(
    'MAKE', 
    type=str, 
    required=True, 
    help='make of the car', 
    location='args')

parser.add_argument(
    'MODEL', 
    type=str, 
    required=True, 
    help='Model of the car', 
    location='args')

parser.add_argument(
    'MILEAGE', 
    type=int, 
    required=True, 
    help='Mileage of the car', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
# Definición de la clase para disponibilización
@ns.route('/')
class PriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_price(args['YEAR','STATE','MAKE','MODEL','MILEAGE'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.